In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sc
import seaborn as sbn


df=pd.read_csv('trainOr.csv')
df_test=pd.read_csv('testOr.csv')
df_test

,Unnamed: 0,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0,0,1,6,2,27.0,0,0.0,0.0,0.0,0.0,0.0
1,1,0,0,5,2,19.0,0,0.0,9.0,0.0,2823.0,0.0
2,2,1,1,2,0,31.0,0,0.0,0.0,0.0,0.0,0.0
3,3,1,0,2,2,38.0,0,0.0,6652.0,0.0,181.0,585.0
4,4,0,0,5,2,20.0,0,10.0,0.0,635.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4272,4272,0,1,6,2,34.0,0,0.0,0.0,0.0,0.0,0.0
4273,4273,0,0,5,2,42.0,0,0.0,847.0,17.0,10.0,144.0
4274,4274,2,1,3,0,29.0,0,0.0,0.0,0.0,0.0,0.0
4275,4275,1,0,3,2,29.0,0,0.0,2680.0,0.0,0.0,523.0


In [2]:
X_train = df.drop("Transported", axis=1)
y_train = df.Transported

X_test = df_test.copy()

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix


In [ ]:
classifiers = {
    "NB": GaussianNB(),
    "SGD": SGDClassifier(),
    "KNN": KNeighborsClassifier(), 
    "LR": LogisticRegression(max_iter=1000), 
    "DT": DecisionTreeClassifier(),
    "RF": RandomForestClassifier(n_estimators=100, max_depth=3, random_state=2),
    "SVM": SVC(),
    "MLP": MLPClassifier(max_iter=1000),
    "XGB": XGBClassifier(),
    "LGBM": LGBMClassifier()
}

results = pd.DataFrame(columns=["Classifier", "Avg_Accuracy", "Avg_F1_Score"])
for name, clf in classifiers.items():
    model = clf
    cv_results = cross_validate(
        model, X_train_scaled, y_train, cv=10,
        scoring=(['accuracy', 'f1'])
    )

    results = results.append({
        "Classifier": name,
        "Avg_Accuracy": cv_results['test_accuracy'].mean(),
        "Avg_F1_Score": cv_results['test_f1'].mean()
    }, ignore_index=True)
    
results["Avg_Overall"] = (results["Avg_Accuracy"] + results["Avg_F1_Score"]) / 2
results = results.sort_values("Avg_Overall", ascending=False)
results

C:\Users\mehmu\AppData\Local\Temp\ipykernel_30284\4057617244.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
C:\Users\mehmu\AppData\Local\Temp\ipykernel_30284\4057617244.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
C:\Users\mehmu\AppData\Local\Temp\ipykernel_30284\4057617244.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
C:\Users\mehmu\AppData\Local\Temp\ipykernel_30284\4057617244.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
C:\Users\mehmu\AppData\Local\Temp\ipykernel_30284\4057617244.py:36: FutureWarnin

,Classifier,Avg_Accuracy,Avg_F1_Score,Avg_Overall
7,MLP,0.789838,0.792123,0.790981
6,SVM,0.789951,0.788059,0.789005
3,LR,0.785116,0.779451,0.782284
1,SGD,0.780402,0.774722,0.777562
2,KNN,0.760391,0.764573,0.762482
5,RF,0.744626,0.761585,0.753105
9,LGBM,0.695033,0.686239,0.690636
4,DT,0.684225,0.688453,0.686339
0,NB,0.701028,0.613684,0.657356
8,XGB,0.648786,0.632168,0.640477


In [53]:
svm = SVC()
params = {
    "tol": (0.1, 0.01, 0.001, 0.0001,0.00001),
    "C": (10.0, 1.0, 0.1, 0.01)
}
clf = GridSearchCV(svm, params, cv=10)
clf.fit(X_train_scaled, y_train)
print("Best hyperparameter:", clf.best_params_)

Best hyperparameter: {'C': 10.0, 'tol': 0.1}


In [ ]:
clf = SVC(C=10,tol=0.1)
clf.fit(X_train_scaled, y_train)
#Score: 0.48421

In [60]:
clf=LogisticRegression()
clf.fit(X_train_scaled, y_train)
#Score: 0.48421

LogisticRegression()

In [61]:
y_pred = clf.predict(X_test_scaled)

y_pred= pd.DataFrame(y_pred)
y_pred.replace({0: False, 1: True}, inplace=True)
y_pred

,0
0,True
1,True
2,False
3,False
4,False
...,...
4272,True
4273,False
4274,False
4275,False


In [62]:
submission = pd.read_csv("sample_submission.csv")
submission["Transported"] = y_pred
submission.to_csv('submission.csv', index=False)

In [63]:
!kaggle competitions submit -c spaceship-titanic -f submission.csv -m ""

Successfully submitted to Spaceship Titanic



  0%|          | 0.00/60.3k [00:00<?, ?B/s]
 13%|█▎        | 8.00k/60.3k [00:00<00:00, 69.9kB/s]
100%|██████████| 60.3k/60.3k [00:01<00:00, 33.8kB/s]
